## 순환신경망 구현 및 학습

In [1]:
import tensorflow as tf

## 하이퍼 파라미터

In [2]:
EPOCHS = 10
NUM_WORDS = 10000

## 모델 정의

In [9]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        self.rnn = tf.keras.layers.SimpleRNN(32) # vanilla
#         self.rnn = tf.keras.layers.LSTM(32) # LSTM
#         self.rnn = tf.keras.layers.GRU(32) # GRU
        self.dense = tf.keras.layers.Dense(2, activation='softmax')
        
    def call(self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

## 학습, 테스트 루프 정의

In [10]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [11]:
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                       value=0,
                                                       padding='pre', # 입력이라 pre로 앞을 0으로 padding
                                                       maxlen=32)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                       value=0,
                                                       padding='pre', # 만약 출력이면 post로 뒤를 0으로 padding 가능
                                                       maxlen=32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [12]:
# Create model
model = MyModel()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [13]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.5598359107971191, Accuracy: 68.83200073242188, Test Loss: 0.45928195118904114, Test Accuracy: 78.26799774169922
Epoch 2, Loss: 0.363840788602829, Accuracy: 84.13599395751953, Test Loss: 0.4758061468601227, Test Accuracy: 77.99199676513672
Epoch 3, Loss: 0.2157832384109497, Accuracy: 91.552001953125, Test Loss: 0.6015660762786865, Test Accuracy: 75.63200378417969
Epoch 4, Loss: 0.08601617813110352, Accuracy: 97.11599731445312, Test Loss: 0.840131938457489, Test Accuracy: 75.26399993896484
Epoch 5, Loss: 0.02998797968029976, Accuracy: 99.08799743652344, Test Loss: 1.0521177053451538, Test Accuracy: 74.7760009765625
Epoch 6, Loss: 0.018359823152422905, Accuracy: 99.45600128173828, Test Loss: 1.216392159461975, Test Accuracy: 73.83200073242188
Epoch 7, Loss: 0.0169590562582016, Accuracy: 99.48799896240234, Test Loss: 1.3226721286773682, Test Accuracy: 74.66799926757812
Epoch 8, Loss: 0.015310412272810936, Accuracy: 99.49600219726562, Test Loss: 1.4023901224136353, Test Acc